In [8]:
import numpy as np
import pandas as pd

In [69]:
df=pd.read_csv('C:/Machine Learning/ML_Datasets/weather.csv')

In [70]:
def find_entropy(df):
    label=df.keys()[-1]
    label_values=df[label].unique()
    entropy=0
    for value in label_values:
        prob=len(df[df[label]==value])/len(df)
        entropy+=-(prob*np.log2(prob))
    return entropy

In [71]:
find_entropy(df)

0.9402859586706309

In [72]:
def find_average_info_entropy(df,attribute):
    target=df.keys()[-1]
    unique_target_values=df[target].unique()
    unique_attribute_values=df[attribute].unique()
    average_info_entropy=0
    for value in unique_attribute_values:
        entropy_subsample=0
        for value1 in unique_target_values:
            num=len(df[attribute][df[attribute]==value][df[target]==value1])
            den=len(df[attribute][df[attribute]==value])
            prob=num/den
            entropy_subsample+=-(prob*np.log2(prob+1e-7))
        weight=den/len(df)
        average_info_entropy+=weight*entropy_subsample
    return average_info_entropy

In [73]:
find_average_info_entropy(df,'Windy')

0.8921586397233878

In [74]:
def find_winner(df):
    attributes=df.keys()[:-1]
    IG=[]
    for attribute in attributes:
        IG.append(find_entropy(df)-find_average_info_entropy(df,attribute))
    print(IG)
    return attributes[np.argmax(IG)]


In [75]:
find_winner(df)

[0.2467500670935654, 0.029222854197929582, 0.15183578990130542, 0.04812731894724309]


'Outlook'

In [76]:
def training(df,tree=None):
    target=df.keys()[-1]
    attribute=find_winner(df)
    if tree is None:
        tree={}
        tree[attribute]={}
    attr_values=df[attribute].unique()
    for value in attr_values:
        sub_df=df[df[attribute]==value].reset_index(drop=True)
        target_values,counts=np.unique(sub_df[target],return_counts=True)
        if (len(counts)==1):
            tree[attribute][value]=target_values[0]
        else:
            tree[attribute][value]=training(sub_df)
    return tree
         

In [77]:
tree=training(df)

[0.2467500670935654, 0.029222854197929582, 0.15183578990130542, 0.04812731894724309]
[2.8853897804381745e-07, 0.5709507964319583, 0.9709507387241655, 0.019973382560951936]
[2.8853897804381745e-07, 0.019973382560951936, 0.019973382560951936, 0.9709507387241655]


In [78]:
import pprint
pprint.pprint(tree)

{'Outlook': {'overcast': 1,
             'rainy': {'Humidity': {'high': 0, 'normal': 1}},
             'sunny': {'Windy': {0: 1, 1: 0}}}}


In [90]:
def prediction(instance,tree):
    for attribute in tree.keys():
        value=instance[attribute]
        sub_tree=tree[attribute][value]
        label=0
        if type(sub_tree) is dict:
            label=prediction(instance,sub_tree)
        else:
            label=sub_tree
            break;
    return label

In [91]:
df1=pd.read_csv('C:/Machine Learning/ML_Datasets/weather_test.csv')
Y_label=[]
for i in range(len(df1)):
    inst =df1.iloc[i,:]
    label = prediction(inst,tree)
    Y_label.append(label)

In [92]:
Y_label

[0, 1]

In [93]:
from sklearn.metrics import classification_report
print(classification_report(df1.iloc[:,-1],Y_label))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



In [87]:
tree.keys()

dict_keys(['Outlook'])